In [ ]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import binary_crossentropy

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def Kernel(x, x0):
    sigma = 0.8
    protonFraction = 0.4
    norm = protonFraction/(np.sqrt(2.*np.pi)*sigma)
    return(norm*np.exp(-(x - x0)**2./(2.*sigma**2.)))

def testDataGen():
    A = 197
    yBeam = 5.36
    slope = 0.5
    sigmaEtas = 0.2
    
    # generate input data
    nBaryons = np.random.randint(0, 2*A)
    randX = np.random.uniform(0, 1, size=nBaryons)
    etasBaryon = 1./slope*np.arcsinh((2.*randX - 1)*np.sinh(slope*yBeam))
    etasArr = np.linspace(-7, 7, 141)
    dNBdetas = np.zeros(len(etasArr))
    norm = 1./(np.sqrt(2.*np.pi)*sigmaEtas)
    for iB in etasBaryon:
        dNBdetas += norm*np.exp(-(etasArr - iB)**2./(2.*sigmaEtas**2.))
    
    # generate test data with convolution with a kernel
    dNpdy = np.zeros(len(etasArr))
    detas = etasArr[1] - etasArr[0]
    for i in range(len(etasArr)):
        dNpdy[i] = sum(Kernel(etasArr, etasArr[i])*dNBdetas)*detas
        
    # generate fake data with random noise
    dNBdetasFake = np.random.uniform(0.0, dNBdetas.max(), size=len(etasArr))
    dNpdyFake = np.random.uniform(0.0, dNpdy.max(), size=len(etasArr))
    return(etasArr, dNBdetas, dNpdy, dNBdetasFake, dNpdyFake)

In [ ]:
x, y1, y2, y3, y4 = testDataGen()
fig = plt.figure()
plt.plot(x, y1)
plt.plot(x, y2)
fig = plt.figure()
plt.plot(x, y3)
plt.plot(x, y4)

In [ ]:
def generateData(size=500):
    print('Generating Data...')
    dataArr = []
    labelArr = []  
    for iev in range(size):
        x, y1, y2, y3, y4 = testDataGen()
        
        dim = len(x)
        
        # real data
        x = np.hstack((y1.reshape(dim, 1), y2.reshape(dim, 1)))
        dataArr.append(x)
        labelArr.append(np.ones((dim, 2)))
        
        # fake data
        x = np.hstack((y3.reshape(dim, 1), y4.reshape(dim, 1)))
        dataArr.append(x)
        labelArr.append(np.zeros((dim, 2)))
    print("done")
    return(np.array(dataArr), np.array(labelArr))

In [ ]:
def trainModel(dim):
    #Sequential model
    model = Sequential()

    #Input layer
    model.add(Dense(16, activation='relu', input_shape=(dim, 2)))

    #Hidden Layer
    model.add(Dense(16, activation='relu', input_shape=(dim, 2)))
    model.add(Dense(4, activation='relu', input_shape=(dim, 2)))

    #Output layer - Binary
    model.add(Dense(2, activation='sigmoid', input_shape=(dim, 2)))

    model.summary()
    
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    return(model)

In [ ]:
def trainModel2(dimShape):
    #Sequential model
    model = Sequential([
        Dense(units=32, activation='relu', input_shape=dimShape),
        Dense(units=64, activation='relu'),
        Dense(units=2, activation='softmax')
    ])
    model.summary()
    
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    return(model)

In [ ]:
data, label = generateData(1000)

In [ ]:
model = trainModel2((data.shape[1], data.shape[2]))

In [ ]:
Nepochs = 30
#Fit will actually train the model.
# X: input of shape (141,2)
# Y: target catagorization, either 1 or 0. Shape (141,2) for consistancy with X
model.fit(
    x=np.array(data),
    y=np.array(label),
    epochs=Nepochs,
    shuffle=True,
    validation_split=0.1,
    use_multiprocessing=True,
    workers=25,
    verbose=2
)